# Payment and charges: a SQL implementation
In Data/payments.csv we have a stream of payment records<br>

In Data/charges.csv we have a stream of charges records<br>

The payments records, that are identified only by the payments date and don't have any reference to the charge record, must be allocated in the order they arrive to the charge records ordered by their bill dates

The final SQL query result is parked in <sql_payment_allocation> pandas dataframe.

Prerequirements:
- Pandas: only to load csv in a table in memory
- SQLite: to have a SQL engine capable to manage my queries without any configuration ( :-) in memory)

In [141]:
import pandas as pd
import sqlite3
# Create a database on the fly in memory
conn = sqlite3.connect(':memory:')


In [142]:
# load <payments> table from the csv file  Data/payments.csv
payments = pd.read_csv('Data/payments.csv',parse_dates=['payment_date'])

payments.to_sql('payments', con=conn, if_exists='replace', index=True, index_label='sequence')
pd.read_sql(
    """
    SELECT *
    FROM payments
--    LIMIT 10
    WHERE customer_id = 1001
    ORDER BY customer_id, payment_date, total_payment
    """,
    con=conn
)

,sequence,customer_id,payment_status,payment_date,rent_payment,utilities_payment,late_fee_payment,total_payment
0,11,1001,PAID,2021-03-28 00:00:00,0,200,0,200
1,12,1001,PAID,2021-04-03 00:00:00,1666,500,0,2166
2,13,1001,PAID,2021-04-19 00:00:00,0,200,0,200
3,14,1001,PAID,2021-04-23 00:00:00,1666,500,0,2166
4,15,1001,PAID,2021-05-29 00:00:00,1666,500,0,2166
5,16,1001,PAID,2021-06-17 00:00:00,0,200,0,200


In [148]:
# load <charges> table from the csv file  Data/charges.csv
charges = pd.read_csv('Data/charges.csv', parse_dates=['scheduled_payment_date'])

charges.to_sql('charges', con=conn, if_exists='replace', index=True, index_label='sequence')
pd.read_sql(
    """
    SELECT *
    FROM charges
    WHERE customer_id = 1001
    ORDER BY customer_id, scheduled_payment_date, scheduled_total_payment
--    LIMIT 10
    """,
    con=conn
)

,sequence,customer_id,scheduled_payment_date,scheduled_rent,scheduled_utilities,scheduled_late_fee,scheduled_total_payment
0,24,1001,2021-04-03 00:00:00,1666,500,0,2166
1,25,1001,2021-04-24 00:00:00,1666,500,0,2166
2,26,1001,2021-05-29 00:00:00,1666,500,0,2166
3,27,1001,2021-06-29 00:00:00,1666,500,0,2166
4,28,1001,2021-07-14 00:00:00,1666,500,0,2166
5,29,1001,2021-07-30 00:00:00,1666,500,0,2166
6,30,1001,2021-08-14 00:00:00,1666,500,0,2166
7,31,1001,2021-08-29 00:00:00,1666,500,0,2166
8,32,1001,2021-09-13 00:00:00,1666,500,0,2166
9,33,1001,2021-09-29 00:00:00,1666,500,0,2166


In [ ]:
# Trasforming payments table into payment segments table 
query = """
WITH 
payment_segments AS (
    SELECT
        customer_id, 
        payment_date,
        total_payment AS segment_size,
        SUM(total_payment) OVER (PARTITION BY customer_id ORDER BY payment_date, total_payment ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS upper_extreme,
        COALESCE(
          SUM(total_payment) OVER (PARTITION BY customer_id ORDER BY payment_date, total_payment ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)
                 , 0) AS lower_extreme
    FROM payments
        )
select customer_id
      ,payment_date
      ,payment_segments.segment_size
      ,payment_segments.lower_extreme
      ,payment_segments.upper_extreme
from  payment_segments 
order by customer_id, lower_extreme
"""
sql_payment_segments = pd.read_sql(query, con=conn)

sql_payment_segments



In [150]:
# Trasforming charges table into charge segments table 
query = """
WITH 
payment_segments AS (
    SELECT
        customer_id, 
        payment_date,
        total_payment AS segment_size,
        SUM(total_payment) OVER (PARTITION BY customer_id ORDER BY payment_date, total_payment ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS upper_extreme,
        COALESCE(
          SUM(total_payment) OVER (PARTITION BY customer_id ORDER BY payment_date, total_payment ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)
                 , 0) AS lower_extreme
    FROM payments
        )
,charge_segments AS (
    SELECT
        customer_id,
        scheduled_payment_date  AS charge_date,
        scheduled_total_payment AS segment_size,
        SUM(scheduled_total_payment) OVER (PARTITION BY customer_id ORDER BY scheduled_payment_date, scheduled_total_payment ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS upper_extreme,
        COALESCE(
          SUM(scheduled_total_payment) OVER (PARTITION BY customer_id ORDER BY scheduled_payment_date, scheduled_total_payment ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)
                 , 0) AS lower_extreme
    FROM charges
        )
select customer_id
      ,charge_date
      ,charge_segments.segment_size
      ,charge_segments.lower_extreme
      ,charge_segments.upper_extreme
from  charge_segments 
order by customer_id, lower_extreme
"""
sql_charge_segments = pd.read_sql(query, con=conn)

sql_charge_segments



,customer_id,charge_date,segment_size,lower_extreme,upper_extreme
0,1001,2021-04-03 00:00:00,2166,0,2166
1,1001,2021-04-24 00:00:00,2166,2166,4332
2,1001,2021-05-29 00:00:00,2166,4332,6498
3,1001,2021-06-29 00:00:00,2166,6498,8664
4,1001,2021-07-14 00:00:00,2166,8664,10830
...,...,...,...,...,...
63,1006,2021-06-29 00:00:00,1050,3550,4600
64,1006,2021-07-14 00:00:00,1050,4600,5650
65,1006,2021-07-29 00:00:00,1050,5650,6700
66,1006,2021-08-14 00:00:00,1050,6700,7750


In [152]:
# Finally, we can cross the payment segments with the charge segments to consider only the overlapping segments.
# Each result row is the overlapping segment that says:
# - the current payment which charge is paying
# - the size of everlapping means how much money is allocated to the charge
# - if the payment segment >= charge segment then the charge is fully paid otherwise is unpaid.   

query = """
WITH 
----------------
payment_segments AS (
----------------
    SELECT
        customer_id, 
        payment_date,
        total_payment AS segment_size,
        SUM(total_payment) OVER (PARTITION BY customer_id ORDER BY payment_date, total_payment ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS upper_extreme,
        COALESCE(
          SUM(total_payment) OVER (PARTITION BY customer_id ORDER BY payment_date, total_payment ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)
                 , 0) AS lower_extreme
    FROM payments
        )
  ---------------
, charge_segments AS (
  ---------------
    SELECT
        customer_id,
        scheduled_payment_date  AS charge_date,
        scheduled_total_payment AS segment_size,
        SUM(scheduled_total_payment) OVER (PARTITION BY customer_id ORDER BY scheduled_payment_date, scheduled_total_payment ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS upper_extreme,
        COALESCE(
          SUM(scheduled_total_payment) OVER (PARTITION BY customer_id ORDER BY scheduled_payment_date, scheduled_total_payment ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)
                 , 0) AS lower_extreme
    FROM charges
        )
  ----------------------------
, payments_overlapping_charges AS (
  ----------------------------
    SELECT
        COALESCE(payment_segments.customer_id, charge_segments.customer_id) AS customer_id
--
        ,payment_segments.payment_date            AS payment_date 
        ,payment_segments.segment_size            AS payment_amount
        ,payment_segments.lower_extreme           AS payment_lower_extreme
        ,payment_segments.upper_extreme           AS payment_upper_extreme
--
        ,charge_segments.charge_date              AS charge_date
        ,charge_segments.segment_size             AS charge_amount
        ,charge_segments.lower_extreme            AS charge_lower_extreme
        ,charge_segments.upper_extreme            AS charge_upper_extreme
--
--      Overlapping segment: 
--
--      Lower is the MAX(charge_segments.lower_extreme, payment_segments.lower_extreme)
--      ,GREATEST(charge_segments.lower_extreme, payment_segments.lower_extreme) --> :-( GREATEST not supported in SQL SERVER, SQLite
        ,CASE WHEN charge_segments.lower_extreme > payment_segments.lower_extreme THEN charge_segments.lower_extreme
                                                                                  ELSE payment_segments.lower_extreme
        END                                       AS overlapping_lower_extreme
--
--      Upper is the MIN(charge_segments.upper_extreme, payment_segments.upper_extreme) 
--      ,LEAST(charge_segments.upper_extreme, payment_segments.upper_extreme)    --> :-( LEAST not supported in SQL SERVER, SQLite
        ,CASE WHEN charge_segments.upper_extreme < payment_segments.upper_extreme THEN charge_segments.upper_extreme
                                                                                  ELSE payment_segments.upper_extreme
        END                                       AS overlapping_upper_extreme  
--
--      Overlapping segment size is the upper - lower 
        ,CASE WHEN charge_segments.upper_extreme < payment_segments.upper_extreme THEN charge_segments.upper_extreme
                                                                                  ELSE payment_segments.upper_extreme
        END                                       -- overlapping_upper_extreme  
        -
        CASE WHEN charge_segments.lower_extreme > payment_segments.lower_extreme THEN charge_segments.lower_extreme
                                                                                  ELSE payment_segments.lower_extreme
        END                                       -- overlapping_lower_extre
                                                  AS allocation_amount -- or overlapping_size
--
--      the status (PAID / UNPAID) of charge and debt or credidit situation in the context of the current payment row
        ,CASE WHEN charge_segments.upper_extreme > payment_segments.upper_extreme THEN 'UNPAID'
              ELSE 'PAID'
        END                                      AS charge_status
        ,CASE WHEN charge_segments.upper_extreme > payment_segments.upper_extreme THEN charge_segments.upper_extreme - payment_segments.upper_extreme
              ELSE NULL 
        END                                      AS current_debt    
        ,CASE WHEN charge_segments.upper_extreme <= payment_segments.upper_extreme THEN payment_segments.upper_extreme - charge_segments.upper_extreme
              ELSE NULL 
        END                                      AS current_credit    
    FROM payment_segments 
    INNER JOIN charge_segments ON (payment_segments.customer_id = charge_segments.customer_id)
    WHERE 
        -- exclude the charge segments without any overlap with payment segments 
        -- NOT (payment_segments.extreme_lower >= charge_segments.extreme_upper OR payment_segments.extreme_upper <= charge_segments.extreme_lower )
        -- it is tautologically equal to
        payment_segments.lower_extreme < charge_segments.upper_extreme AND payment_segments.upper_extreme > charge_segments.lower_extreme 
        )

SELECT customer_id
       ,payments_overlapping_charges.payment_date 
       ,payments_overlapping_charges.payment_amount
       ,payments_overlapping_charges.allocation_amount
       ,payments_overlapping_charges.charge_date
       ,payments_overlapping_charges.charge_amount
       ,payments_overlapping_charges.charge_status
       ,payments_overlapping_charges.current_debt
       ,payments_overlapping_charges.current_credit
FROM payments_overlapping_charges 
ORDER BY customer_id
        ,payments_overlapping_charges.payment_date
        ,payments_overlapping_charges.payment_amount
"""
sql_payment_allocation = pd.read_sql(query, con=conn)
sql_payment_allocation


,customer_id,payment_date,payment_amount,allocation_amount,charge_date,charge_amount,charge_status,current_debt,current_credit
0,1001,2021-03-28 00:00:00,200,200,2021-04-03 00:00:00,2166,UNPAID,1966.0,NaN
1,1001,2021-04-03 00:00:00,2166,1966,2021-04-03 00:00:00,2166,PAID,NaN,200.0
2,1001,2021-04-03 00:00:00,2166,200,2021-04-24 00:00:00,2166,UNPAID,1966.0,NaN
3,1001,2021-04-19 00:00:00,200,200,2021-04-24 00:00:00,2166,UNPAID,1766.0,NaN
4,1001,2021-04-23 00:00:00,2166,1766,2021-04-24 00:00:00,2166,PAID,NaN,400.0
5,1001,2021-04-23 00:00:00,2166,400,2021-05-29 00:00:00,2166,UNPAID,1766.0,NaN
6,1001,2021-05-29 00:00:00,2166,1766,2021-05-29 00:00:00,2166,PAID,NaN,400.0
7,1001,2021-05-29 00:00:00,2166,400,2021-06-29 00:00:00,2166,UNPAID,1766.0,NaN
8,1001,2021-06-17 00:00:00,200,200,2021-06-29 00:00:00,2166,UNPAID,1566.0,NaN
9,1002,2020-12-28 00:00:00,2083,2083,2020-12-27 00:00:00,2083,PAID,NaN,0.0
